## 実習

### ライブラリのimport

In [1]:
# 必要なライブラリのimport
import numpy as np

# 文章データ分析用
import glob
import collections
from sklearn.feature_extraction import DictVectorizer

### データの読み込み

In [2]:
# ダウンロードしたファイルを解凍する
!tar xvf 20030228_easy_ham.tar.bz2
!tar xvf 20030228_hard_ham.tar.bz2
!tar xvf 20030228_spam.tar.bz2

x easy_ham/
x easy_ham/00001.7c53336b37003a9286aba55d2945844c
x easy_ham/00002.9c4069e25e1ef370c078db7ee85ff9ac
x easy_ham/00003.860e3c3cee1b42ead714c5c874fe25f7
x easy_ham/00004.864220c5b6930b209cc287c361c99af1
x easy_ham/00005.bf27cdeaf0b8c4647ecd61b1d09da613
x easy_ham/00006.253ea2f9a9cc36fa0b1129b04b806608
x easy_ham/00007.37a8af848caae585af4fe35779656d55
x easy_ham/00008.5891548d921601906337dcf1ed8543cb
x easy_ham/00009.371eca25b0169ce5cb4f71d3e07b9e2d
x easy_ham/00010.145d22c053c1a0c410242e46c01635b3
x easy_ham/00011.fbcde1b4833bdbaaf0ced723edd6e355
x easy_ham/00012.48a387bc38d1316a6f6b49e8c2e43a03
x easy_ham/00013.81c34741dbed59c6dde50777e27e7ea3
x easy_ham/00014.cb20e10b2bfcb8210a1c310798532a57
x easy_ham/00015.4d7026347ba7478c9db04c70913e68fd
x easy_ham/00016.ef397cef16f8041242e3b6560e168053
x easy_ham/00017.08ef2d89f14cf7e2a458b80697eb1837
x easy_ham/00018.6fee38026193b5adde4b56892a6f14bc
x easy_ham/00019.5322cb10d8819b39499924d852819c27
x easy_ham/00020.d10651e31fcb92630c622

In [3]:
# それぞれのフォルダのファイル名が「0」から始まるファイルの一覧を取得
easy_ham_files = glob.glob("./easy_ham/0*")
hard_ham_files = glob.glob("./hard_ham/0*")
spam_files = glob.glob("./spam/0*")

In [4]:
# それぞれのファイル数を確認
print(len(easy_ham_files))
print(len(hard_ham_files))
print(len(spam_files))

2500
250
500


In [5]:
# ファイル名をまとめたリストを用意
filenames = easy_ham_files + hard_ham_files + spam_files
len(filenames)

3250

In [6]:
# エンコーディング用定数（以下で繰り返し指定するため定数にしておく）
ENCODING = 'latin-1'

In [7]:
# 最初のファイルの内容を確認
with open(filenames[0], 'r', encoding=ENCODING) as f:
    text = f.read()
    print(text)

From exmh-workers-admin@redhat.com  Thu Aug 22 12:36:23 2002
Return-Path: <exmh-workers-admin@spamassassin.taint.org>
Delivered-To: zzzz@localhost.netnoteinc.com
Received: from localhost (localhost [127.0.0.1])
	by phobos.labs.netnoteinc.com (Postfix) with ESMTP id D03E543C36
	for <zzzz@localhost>; Thu, 22 Aug 2002 07:36:16 -0400 (EDT)
Received: from phobos [127.0.0.1]
	by localhost with IMAP (fetchmail-5.9.0)
	for zzzz@localhost (single-drop); Thu, 22 Aug 2002 12:36:16 +0100 (IST)
Received: from listman.spamassassin.taint.org (listman.spamassassin.taint.org [66.187.233.211]) by
    dogma.slashnull.org (8.11.6/8.11.6) with ESMTP id g7MBYrZ04811 for
    <zzzz-exmh@spamassassin.taint.org>; Thu, 22 Aug 2002 12:34:53 +0100
Received: from listman.spamassassin.taint.org (localhost.localdomain [127.0.0.1]) by
    listman.redhat.com (Postfix) with ESMTP id 8386540858; Thu, 22 Aug 2002
    07:35:02 -0400 (EDT)
Delivered-To: exmh-workers@listman.spamassassin.taint.org
Received: from int-mx1.corp

In [8]:
# 最後のファイルの内容を確認
with open(filenames[-1], 'r', encoding=ENCODING) as f:
    text = f.read()
    print(text)

From email@hkem.com  Tue Dec  3 11:46:36 2002
Return-Path: <email@hkem.com>
Delivered-To: yyyy@localhost.spamassassin.taint.org
Received: from localhost (jalapeno [127.0.0.1])
	by jmason.org (Postfix) with ESMTP id F161516F16
	for <jm@localhost>; Tue,  3 Dec 2002 11:46:31 +0000 (GMT)
Received: from jalapeno [127.0.0.1]
	by localhost with IMAP (fetchmail-5.9.0)
	for jm@localhost (single-drop); Tue, 03 Dec 2002 11:46:32 +0000 (GMT)
Received: from hkem.com ([61.236.128.64]) by dogma.slashnull.org
    (8.11.6/8.11.6) with ESMTP id gB3Be9803237 for <jm@jmason.org>;
    Tue, 3 Dec 2002 11:40:11 GMT
Message-Id: <200212031140.gB3Be9803237@dogma.slashnull.org>
From: email@hkem.com
Subject: HK Email marking !
To: yyyy@spamassassin.taint.org
Content-Type: text/html;charset="GB2312"
Reply-To: email@hkem.com
Date: Tue, 3 Dec 2002 19:41:57 +0800
X-Priority: 3
X-Mailer: Microsoft Outlook Express 5.00.2615.200

<STYLE type="text/css">
<!--
P{
  font-size : 12px;
  color : navy;
}
-->
</STYLE>
</HEAD>


### データの前処理 - 形態素解析と特徴量の作成

In [9]:
# 文字列のなかで使われている単語ごとの数を返す
def get_word_count(text, min_length=3):
    # ノイズの除去：不要と思われる文字を除去する
    for ch in ".,:;!?-+*/=()[]{}<>~^#$@%&'\"_0123456789":
        text = text.replace(ch, ' ')

    # 形態素解析：文章を単語に分割
    _words = text.strip().split()

    # 表記のゆれの補正：
    # 単語のリストを受け取り、指定された文字数以上の単語だけをすべて小文字にして返す
    _words = [_word.lower() for _word in _words if len(_word) >= min_length]

    # collections.Counterの戻り値は辞書型のサブクラス
    _count = collections.Counter(_words)

    # 辞書型に変換して返す
    return dict(_count)

In [10]:
# 最初のファイルでテスト
with open(filenames[0], 'r', encoding=ENCODING) as f:
    text = f.read()

get_word_count(text)

{'from': 16,
 'exmh': 25,
 'workers': 23,
 'admin': 4,
 'redhat': 12,
 'com': 22,
 'thu': 12,
 'aug': 13,
 'return': 1,
 'path': 1,
 'spamassassin': 23,
 'taint': 23,
 'org': 24,
 'delivered': 2,
 'zzzz': 4,
 'localhost': 9,
 'netnoteinc': 2,
 'received': 10,
 'phobos': 2,
 'labs': 1,
 'postfix': 3,
 'with': 9,
 'esmtp': 6,
 'for': 9,
 'edt': 3,
 'imap': 1,
 'fetchmail': 1,
 'single': 1,
 'drop': 1,
 'ist': 1,
 'listman': 12,
 'dogma': 1,
 'slashnull': 1,
 'mbyrz': 1,
 'localdomain': 1,
 'int': 5,
 'corp': 4,
 'mail': 1,
 'mby': 2,
 'smtp': 2,
 'ratree': 2,
 'psu': 3,
 'mbihl': 1,
 'delta': 6,
 'coe': 1,
 'mbwel': 1,
 'ict': 3,
 'munnari': 4,
 'mbqpw': 1,
 'robert': 1,
 'elz': 1,
 'kre': 2,
 'chris': 2,
 'garrigues': 2,
 'cwg': 2,
 'dated': 2,
 'deepeddy': 8,
 'subject': 7,
 'new': 1,
 'sequences': 1,
 'window': 1,
 'reply': 1,
 'tmda': 6,
 'vircio': 6,
 'references': 1,
 'mime': 1,
 'version': 5,
 'content': 1,
 'type': 1,
 'text': 1,
 'plain': 1,
 'charset': 1,
 'ascii': 1,
 'message

In [11]:
# 単語ごとの数のリスト
word_count_data = []

# すべてのファイルに対して実施
for filename in filenames:
    with open(filename, 'r', encoding=ENCODING) as f:
        text = f.read()
        count = get_word_count(text)
        word_count_data.append(count)

In [12]:
# リストの長さ
len(word_count_data)

3250

In [13]:
# リストの最初の内容
word_count_data[0]

{'from': 16,
 'exmh': 25,
 'workers': 23,
 'admin': 4,
 'redhat': 12,
 'com': 22,
 'thu': 12,
 'aug': 13,
 'return': 1,
 'path': 1,
 'spamassassin': 23,
 'taint': 23,
 'org': 24,
 'delivered': 2,
 'zzzz': 4,
 'localhost': 9,
 'netnoteinc': 2,
 'received': 10,
 'phobos': 2,
 'labs': 1,
 'postfix': 3,
 'with': 9,
 'esmtp': 6,
 'for': 9,
 'edt': 3,
 'imap': 1,
 'fetchmail': 1,
 'single': 1,
 'drop': 1,
 'ist': 1,
 'listman': 12,
 'dogma': 1,
 'slashnull': 1,
 'mbyrz': 1,
 'localdomain': 1,
 'int': 5,
 'corp': 4,
 'mail': 1,
 'mby': 2,
 'smtp': 2,
 'ratree': 2,
 'psu': 3,
 'mbihl': 1,
 'delta': 6,
 'coe': 1,
 'mbwel': 1,
 'ict': 3,
 'munnari': 4,
 'mbqpw': 1,
 'robert': 1,
 'elz': 1,
 'kre': 2,
 'chris': 2,
 'garrigues': 2,
 'cwg': 2,
 'dated': 2,
 'deepeddy': 8,
 'subject': 7,
 'new': 1,
 'sequences': 1,
 'window': 1,
 'reply': 1,
 'tmda': 6,
 'vircio': 6,
 'references': 1,
 'mime': 1,
 'version': 5,
 'content': 1,
 'type': 1,
 'text': 1,
 'plain': 1,
 'charset': 1,
 'ascii': 1,
 'message

In [14]:
# 行列に変換してdatasetに格納する
vec = DictVectorizer()
dataset = vec.fit_transform(word_count_data)

In [15]:
# datasetを表示
dataset

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 756721 stored elements and shape (3250, 121830)>

In [16]:
# 各列に対応した単語を取得
vec.get_feature_names_out()

array(['\x01\x14www', '\x13c\x14', '\x14\x01\x15', ...,
       'þ¿ú²\x9fë\xadç\x9f¢¸', 'ÿ\x8aza¢x\x9cýê',
       'ÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿó\x86'], dtype=object)

In [17]:
# 型の確認
type(dataset)

scipy.sparse._csr.csr_matrix

In [18]:
# 0でない要素だけ保持していることを確認
print(dataset[0])

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 177 stored elements and shape (1, 121830)>
  Coords	Values
  (0, 769)	1.0
  (0, 1381)	1.0
  (0, 1598)	4.0
  (0, 2443)	1.0
  (0, 4099)	2.0
  (0, 5160)	1.0
  (0, 5493)	1.0
  (0, 6131)	13.0
  (0, 8218)	1.0
  (0, 8219)	1.0
  (0, 10599)	1.0
  (0, 11713)	1.0
  (0, 12830)	1.0
  (0, 14313)	1.0
  (0, 14650)	2.0
  (0, 16052)	1.0
  (0, 16080)	1.0
  (0, 16271)	22.0
  (0, 16299)	2.0
  (0, 16320)	3.0
  (0, 16478)	1.0
  (0, 16946)	1.0
  (0, 17182)	4.0
  (0, 17764)	1.0
  (0, 18640)	1.0
  :	:
  (0, 97130)	1.0
  (0, 97395)	3.0
  (0, 97422)	15.0
  (0, 97489)	1.0
  (0, 97625)	1.0
  (0, 97653)	3.0
  (0, 97801)	12.0
  (0, 97991)	1.0
  (0, 98279)	1.0
  (0, 98504)	6.0
  (0, 98715)	1.0
  (0, 100666)	1.0
  (0, 103125)	2.0
  (0, 103772)	2.0
  (0, 105348)	5.0
  (0, 105368)	1.0
  (0, 105974)	6.0
  (0, 109276)	1.0
  (0, 109691)	1.0
  (0, 110069)	1.0
  (0, 110202)	9.0
  (0, 110215)	1.0
  (0, 110901)	23.0
  (0, 110914)	1.0
  (0, 121017)	4.0


### 機械学習の実施

In [19]:
# 必要なライブラリの追加import
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [20]:
# 目的変数の用意
Y = [0]*2500 + [0]*250 + [1]*500

In [21]:
# データの分割
X_train, X_test, Y_train, Y_test = train_test_split(dataset, Y, test_size=0.3, random_state=0)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=0.3, random_state=0)

In [22]:
# ロジスティック回帰の実施
logistic_model = LogisticRegression(max_iter=2000)
logistic_model.fit(X_train, Y_train)
Y_pred = logistic_model.predict(X_valid)

# モデルの評価
print(classification_report(Y_valid, Y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       587
           1       0.93      0.97      0.95        96

    accuracy                           0.99       683
   macro avg       0.96      0.98      0.97       683
weighted avg       0.99      0.99      0.99       683



### 学習結果の考察

In [23]:
# 比較
print("datasetの大きさ:", dataset.shape)
print("単語辞書の長さ:", len(vec.get_feature_names_out()))
print("モデルのパラメータ数:", logistic_model.coef_.shape)

datasetの大きさ: (3250, 121830)
単語辞書の長さ: 121830
モデルのパラメータ数: (1, 121830)


In [24]:
import pandas as pd

# 単語とパラメータの表を作成
coef_df = pd.DataFrame({
    'feature_name': vec.get_feature_names_out(),
    'coef': logistic_model.coef_[0]
})

# 先頭5行を表示
coef_df.head()

,feature_name,coef
0,www,0.000000
1,c,0.000000
2,,0.000000
3,\\\\,-0.000068
4,\\\x\,-0.002778


In [25]:
# スパムらしさに対する寄与が高い一覧
coef_df.sort_values('coef', ascending=False).head(20)

,feature_name,coef
121017,zzzz,0.722679
109225,webnote,0.359094
116526,your,0.330730
92541,smtp,0.321899
121023,zzzzason,0.305696
116508,you,0.271698
10313,body,0.242498
67935,newnamedns,0.223681
1465,adclick,0.221490
13989,cfm,0.219652


In [26]:
# ハムらしさに対する寄与が高い一覧
coef_df.sort_values('coef').head(20)

,feature_name,coef
87944,rssfeeds,-0.406941
67817,netnoteinc,-0.379228
19474,date,-0.364888
97983,tim,-0.268596
71419,oct,-0.259067
16290,comcast,-0.242844
93154,spamassassin,-0.239645
11872,but,-0.233872
93155,spambayes,-0.228669
72757,one,-0.228455
